# TOP

## Sections

* [%ruby](#%ruby) startup time (restarrt?)
* [HD.py](#HD.py) (%debug)
* [FileMgr.py](#FileMgr.py) (%debug)
* [Money.py](#Money.py) (%debug // Excel)
* [MOOCS.py](#MOOCS.py) (%debug // Excel)
* [Magics](#Magics)
* [APIs - Docker](#APIS_DOCKER) (%debug)
* [APIs - OpenStack](#APIS_OSTACK) (%debug)

## TO ADD:
* Other APIs: Oneview / Redfish / iLo
* GANTT (from MOOCs ... above)
* Nutrition.py/js




[TOP](#TOP)
---
# HD.py

In [ ]:
%cd d:/z/Dropbox/z/ArchiveIndex/Hardware/
#!ls -al: cygwin ls
#%ls : DOS dir

In [ ]:
#%run ./HardDrives.py

#import HardDrives
import importlib
#imp.reload( HardDrives)
importlib.reload( HardDrives)

In [ ]:
#print(len(devices))
#print("XX")
#sys.argv=[]
devices = HardDrives.main()
#print("OK?")
#print(str(devices))

In [ ]:
print(len(devices))


# FileMgr.py (dups ... cleversync)

[TOP](#TOP)
---
# Money.py

In [2]:
%cd d:/z/Dropbox/z/data/Money/DATA/
    
!ls -altr

d:\z\Dropbox\z\data\Money\DATA
total 1769
-rwxr-xr-x 1 mjbright 10513  12783 Jan 26  2013 money.rb.win64
-rw-r--r-- 1 mjbright 10513    607 Jan 27  2013 UK_GBP_ICI_SHARES.mon
-rw-r--r-- 1 mjbright 10513   7991 Jan 28  2013 MONEY.txt
-rwxr-xr-x 1 mjbright 10513  22340 Jan 29  2013 money.rb~
-rw-r--r-- 1 mjbright 10513   7432 Jan 29  2013 FR_EU_Amex-CC.mon
-rw-r--r-- 1 mjbright 10513  22638 Jan 29  2013 FR_EU_BDirecte-VISA.mon
-rw-r--r-- 1 mjbright 10513   5514 Jan 29  2013 UK_GBP_Access-CC.mon
-rw-r--r-- 1 mjbright 10513    925 Jan 29  2013 UK_GBP_MandS_CC.mon
-rw-r--r-- 1 mjbright 10513    390 Jan 29  2013 FR_EU_ING_SA.mon
-rw-r--r-- 1 mjbright 10513  26624 Jan 29  2013 UK_GBP_Barclay-VISA.mon
-rw-r--r-- 1 mjbright 10513  13681 Jan 29  2013 UK_GBP_BarclayAccess-CC.mon
-rw-r--r-- 1 mjbright 10513  31873 Jan 29  2013 UK_GBP_Barclays_CA.mon
-rw-r--r-- 1 mjbright 10513  11774 Jan 29  2013 UK_GBP_ChaseM-VISA.mon
-rw-r--r-- 1 mjbright 10513   6013 Jan 29  2013 UK_GBP_SaveP-VISA.mon
-rw-r--r-

In [21]:
#import glob
import os
import sys
import csv

import re

import hashlib

import datetime

CSV_SUBDIR="BanquePostale.CSV"

files = [ CSV_SUBDIR+"/"+file for file in os.listdir(CSV_SUBDIR) if file.endswith(".csv") ]
#files = [ file for file in glob.glob(CSV_SUBDIR + "/*.csv") ]
#rdf = [ (root,dirs,files) for root, dirs, files in os.walk(CSV_SUBDIR) ]
#print(str(files))

comptes = {
    '0992100C028': [ 'CJointe', 'FR_EU_LaPoste_CJointe' ],
    '1063284B028': [ 'CCP',     'FR_EU_LaPoste_CCP' ]
}


REGEXES = {
    '^COTISATION TRIMESTRIELLE DE VOTRE FORMULE DE COMPTE':     [ 'LaPoste', 'Bank;Charges', '' ],
    '^FRAIS DE (.+)':                                           [ 'LaPoste', 'Bank;Charges', '' ],
    '^REGULARISATION AU CREDIT VIREMENT DE LA BANQUE POSTALE':  [ 'LaPoste', 'REMBOURSEMENT Bank;Charges', '$1' ],
    '^VIREMENT POUR MR BRIGHT MICHAEL COMPTE 20041010171063284B02822 (.+)': [ 'LaPoste', '[Mike_CCP]', '$1' ],
    '^VIREMENT POUR MME BRIGHT PAULINA': [ 'LaPoste', '[Pao_CCP]', '$1' ],
    '^VIREMENT DE HP CTRE COMP':         [ 'HP', 'Salary__VMTRECU', '' ],
    '^VIREMENT PERMANENT POUR MR BRIGHT MICHAEL COMPTE FR6710011000206742101456Y': [ 'LaPoste', '[PEL]', '' ],
    '^PRELEVEMENT DE POITIERS.*Cachemire': [ 'LaPoste', 'Assurance;Vie', 'AssuranceVie Cachemire2' ],
    '^PRELEVEMENT DE POITIERS.*VIVACCIO': [ 'LaPoste', 'Assurance;Vie', 'AssuranceVie Vivaccio' ],
    '^PRELEVEMENT DE EDF': [ 'EDF', 'FIX Household;Electricity', '' ],
    '^PRELEVEMENT DE GDF': [ 'GDF', 'FIX Household;Gas', '' ],
    '^PRELEVEMENT DE FONDATION WWF': [ 'WWF', 'FIX Charity', 'World Wildlife Fund' ],
    '^PRELEVEMENT DE DIRECTION GENERAL.*-124.0': [ 'Impots', 'FIX Impots;Habitation', "Taxe d'Habitation" ],
    '^PRELEVEMENT DE DIRECTION GENERAL.*-550.0': [ 'Impots', 'FIX Impots;Revenue', 'Impots sur la Revenue' ],
    '^PRELEVEMENT DE DIRECTION GENERAL.*-104.0': [ 'Impots', 'FIX Impots;Fonciere', 'Taxe Fonciere' ],
    '^PRELEVEMENT DE ECHEANCE PRET': [ 'LaPoste', 'FIX Household;Pret', 'UNKNOWN' ],
    '^PRELEVEMENT DE MATMUT ROUEN': [ 'MatMut', 'FIX Assurance;Voiture', 'Assurance Voiture - Paulina/DDY' ],
    '^PRELEVEMENT DE Free Telecom': [ 'Free Telecom', 'FIX Phone;Internet', 'Abonnement Phone/Internet' ],
    '^PRELEVEMENT DE SDC LE MINERVE': [ 'SDC Le Minerve', 'FIX Household;Charges', 'Syndicat' ],
            
    '^VIREMENT DE MME (GUTIERREZ PAULIN|BRIGHT PAULINA) PARTICIPATION MENSUELLE': [ 'LaPoste', '[Pao]', 'Monthly Contrib' ],
    '^VIREMENT DE MME MME (GUTIERREZ PAULIN|BRIGHT PAULINA) (.+)':                [ 'LaPoste', '[Pao]', '$2' ],
    '^VIREMENT DE (M. MICHAEL JOHN BRIG|BRIGHT MICHAEL JOHN) BILLSFOODTAXES': [ 'Axa', '[Mike_Axa]', 'Monthly Contrib-BillsFoodTaxes' ],
    '^VIREMENT DE (M. MICHAEL JOHN BRIG|BRIGHT MICHAEL JOHN) ASSURANCEVIE':   [ 'Axa', '[Mike_Axa]', 'for AssuranceVie Vivaccio' ],
    '^VIREMENT DE (M. MICHAEL JOHN BRIG|BRIGHT MICHAEL JOHN|MR\s+BRIGHT MICHAEL) (.+)': [ 'Axa', '[Mike_Axa]', '$2' ],
    '^VIREMENT DE VIVACCIO': [ 'LaPoste', 'FIX Assurance;Vie', 'AssuranceVie Vivaccio' ],
    '^VIREMENT DE MME GUTIERREZ PAULIN MARCELA REMBOURSEMENT': [ 'Marcela', 'Remboursement', '' ],
        
    'CHEQUE N\s+(\d+)':  [ 'Unknown', 'Cheque', 'ChequeNo $1'],
    'REMISE DE CHEQUES': [ 'Unknown', 'Cheque', '' ],
    'CREDIT':            [ 'C4', 'REMBOURSEMENT Household', ''],
    '(CARTE \S+)\s+(\S+) A (\d+H\d+) RETRAIT DAB (.+)$': [ 'Bank', 'DAB;CashOut', '$2, $3, $4' ],

    '^ACHAT CB.*AMAZON S.E..* CARTE (NUMERO|NO)\s+(276|268)$': [ 'Amazon', 'Misc', 'Amazon S.E. (Marketplace?)' ],
    '^ACHAT CB.*GOOGLE.* CARTE (NUMERO|NO)\s+(276|268)$':      [ 'Google', 'Misc', '' ],
    '^ACHAT CB.*LA POSTE.* CARTE (NUMERO|NO)\s+(276|268)$': [ 'LaPoste', 'Misc', '' ],
        
    '^ACHAT CB.*.*PARIS DELICE.* CARTE (NUMERO|NO)\s+(276|268)$': [ 'Paris Delice', 'Food', 'Patisserie, Grenoble' ],
    '^ACHAT CB.*(R S\d+VERSAI).* CARTE (NUMERO|NO)\s+(276|268)$': [ 'SNCF', 'Food', '' ],
        
    '^ACHAT CB.*LP ST EGREVE.* CARTE (NUMERO|NO)\s+(276|268)$': [ 'Leader Price', 'Household;Food', '' ],
    '^ACHAT CB.*GEANT.* CARTE (NUMERO|NO)\s+(276|268)$': [ 'Geant Casino', 'Household;Food', '' ],
    '^ACHAT CB.*SM CASINO.* CARTE (NUMERO|NO)\s+(276|268)$': [ 'Casino', 'Household;Food', '' ],
    '^ACHAT CB.*CARREFOUR.* CARTE (NUMERO|NO)\s+(276|268)$': [ 'C4', 'Household;Food', '' ],
    '^ACHAT CB.*INTERMARCHE SE.* CARTE (NUMERO|NO)\s+(276|268)$': [ 'Intermarche', 'Household;Food', '' ],
    '^ACHAT CB.*FOURNIL PERE R.* CARTE (NUMERO|NO)\s+(276|268)$': [ 'Fournil Pere Richard', 'Household;Food', '' ],
        
    '^ACHAT CB.*(AMORINO|HAAGEN *DAZS|LA NEIGE).* CARTE (NUMERO|NO)\s+(276|268)$': [ '$1', 'Food;IceCream', '' ],
        
    '^ACHAT CB.* CARTE (NUMERO|NO)\s+(276|268)$': [ 'Commercant', 'Misc', '' ],
    '^ACHAT CB.* CARTE (NUMERO|NO)\s+(276|268)$': [ 'Commercant', 'Misc', '' ],
    '^ACHAT CB.* CARTE (NUMERO|NO)\s+(276|268)$': [ 'Commercant', 'Misc', '' ],
    '^ACHAT CB.* CARTE (NUMERO|NO)\s+(276|268)$': [ 'Commercant', 'Misc', '' ],

    '^ACHAT CB.*(LE MEZZE|BRAS. DES FLEU|AU BUREAU|MCDONALDS|DRAGON DE CHIN|RED HIPPO|FLUNCH|HIPPOPOTAMUS).* CARTE (NUMERO|NO)\s+(276|268)$': [ '$1', 'Resto', '' ],
    '^ACHAT CB.*(LE RELAIS DU C|PAL. DE LA BIE/)': [ '$1', 'Resto', '' ],
    '^ACHAT CB.*(KIOTORI|LA PATACCA|LES PATIOS|LE TARMAC)': [ '$1', 'Resto', 'Paris' ],

    # $COMMERCANT="Brasserie des Fleurs"; $INFO="Grenoble, Pl. Grenette";
    # $COMMERCANT="Au Bureau"; $INFO="Grenoble, Pl. Grenette";

    '^ACHAT CB.*(AREA)': [ '$1', 'Car;Peage', '' ],
    '^ACHAT CB.*(PARCVAUCANSON)': [ '$1', 'Car;Parking', '' ],
    '^ACHAT CB.*(ESSFLOREAL)': [ 'Casino', 'Car;Petrol', '' ],

    '^ACHAT CB.*(HOTEL $1)': [ 'Hotel $1', 'Hotel', '' ],

    '^ACHAT CB.*(CASTORAMA|LEROY MERLIN)': [ '$1', 'Household;DIY', '' ],

    '^ACHAT CB.*(TABAC PRESSE)': [ 'Tabac Presse', 'Household;Misc', '' ],
    '^ACHAT CB.*(MAISONS DU MON)': [ 'Maisons du Monde', 'Household;Misc', '' ],
}

#################################################################################################
# Functions:

def die(msg):
    print("die: " + msg)
    sys.exit(1)
    
def debug(verbose, level, msg):
    if verbose >= level:
        print("DEBUG <" + msg + ">")

def parseSolde(field):
    parts = []
    verbose=1
    
    if "," in field:
        parts = field.split(",")
    if "." in field:
        parts = field.split(".")
        
    if len(parts) != 2:
        die("Bad numeric value <" + field + ">")

    #TODO: Do a regex tocheck format here
    #debug(verbose, 1, "field={}".format(field))
    euros=float(parts[0])
    cents=float(parts[1])
    #debug(verbose, 1, "euros={}".format(euros))
    #debug(verbose, 1, "cents={}".format(cents))
    val = euros + (0.01 * cents)
    return val
        
def getDateNumber(dateStr):
    # yyyy-mm-dd
    (year, month, day) = dateStr.split("-")
    year = int(year)
    month = int(month)
    day = int(day)
    if year < 2015 or year > 2015:
        die("Year {} out of range".format(year))
    if month < 1 or month > 12:
        die("Month {} out of range".format(month))
    if day < 1 or day > 31:
        die("Day {} out of range".format(day))
    
    BASE_YEAR=1980
    
    return day + (31*month) + (366*(year-BASE_YEAR))

def parseMMDDYYtoISO(dateStr):
    verbose = 1
    
    date = dateStr.split("/")
    date.reverse()
    date = "-".join(date)
    #debug(verbose, 1, "date=" + str(date))
    dateNum = getDateNumber(date)
    return date, dateNum
                   
def readCSV(file):
    #print(file)
    compteName=None
    date=None
    solde=None
    entries=[]
    
    localHashes={}
    
    verbose=1
    debug(verbose, 1, "Parsing CSV file <" + file + ">")
    
    minDate=None
    minDatenum=None
    maxDate=None
    maaxDatenum=None
    
    line=0
    with open(file) as csvfile:
        readCSV = csv.reader(csvfile, delimiter=';')
        
        inHeader = True
        
        for row in readCSV:
            line += 1
            if len(row) == 0:
                continue
            
            if inHeader:
                if "Type" in row[0]:
                    continue

                if "Compte tenu en" in row[0]:
                    continue

                if "Solde (FRANCS)" in row[0]:
                    continue
                    
                if row[0].find("Num") == 0 and "Compte" in row[0]:
                    compte=row[1]
                    if not compte in comptes:
                        sys.exit(1)
                        
                    compteName = comptes[compte][0]
                    debug(verbose, 2, "compte=" + compteName)
                    continue
                    
                if row[0].find("Date") == 0 and date == None:
                    #date = row[1].split("/")
                    #date.reverse()
                    #date = "-".join(date)
                    #dateNum = getDateNumber(date)
                    date, dateNum = parseMMDDYYtoISO(row[1])
                    debug(verbose, 2, "STATEMENT date=" + str(date))
                    continue
                    
                if row[0].find("Solde") == 0 and "EUROS" in row[0]:
                    solde = parseSolde(row[1])
                    debug(verbose, 2, "solde=" + str(solde))
                    continue
                    
                if "Date" in row[0] and "Libell" in row[1]:
                    inHeader = False
                    continue
                    
                die("Bad syntax in header row:" + str(row))
                
            # Date;Libelle;Montant(EUROS);Montant(FRANCS)
            md5 = hashlib.md5( ("".join(row)).encode() ).hexdigest()
            date, dateNum = parseMMDDYYtoISO(row[0])
            row[0] = date
            if not minDate:
                minDate = date
                minDatenum = dateNum
                
            if not maxDate:
                #print("Setting initial maxDate to " + date)
                maxDate = date
                maxDatenum = dateNum
                
            if dateNum < minDatenum:
                minDatenum = dateNum 
                minDate    = date
                
            if dateNum > maxDatenum:
                maxDatenum = dateNum 
                maxDate    = date
                    
            row[2] = parseSolde(row[2])
                    
            #print("File {}/line {} md5:{} row:{}".format(file, line, md5, row))
            debug(verbose, 2, ("Line {} md5:{} row:{}".format(line, md5, row)))
            if md5 in localHashes:
                die("Already seen this hash in Line {} md5:{} row:{}".format(line, md5, row[:2]))

            entries.append( (md5, row) )
            #print(md5 + ": " + str(row))
            continue            
            #break
            #sys.exit(0)

    entries.reverse()
    return (compte, date, minDate, maxDate, solde, entries)

def reverse_numeric(x, y):
        return y - x
    
def cmp_to_key(mycmp):
    'Convert a cmp= function into a key= function'
    class K:
        def __init__(self, obj, *args):
            self.obj = obj
        def __lt__(self, other):
            return mycmp(self.obj, other.obj) < 0
        def __gt__(self, other):
            return mycmp(self.obj, other.obj) > 0
        def __eq__(self, other):
            return mycmp(self.obj, other.obj) == 0
        def __le__(self, other):
            return mycmp(self.obj, other.obj) <= 0
        def __ge__(self, other):
            return mycmp(self.obj, other.obj) >= 0
        def __ne__(self, other):
            return mycmp(self.obj, other.obj) != 0
    return K

def compareMinMaxDates(range1, range2):
    if range1[3] < range2[3]: # minDate on range1 is earlier
        return 1
    
    if range1[3] > range2[3]: # minDate on range1 is later
        return -1
    
    if range1[4] < range2[4]: # maxDate on range1 is earlier
        return 1
    
    if range1[4] > range2[4]: # maxDate on range1 is later
        return -1
    
    return 0 # same range
    
#################################################################################################
# Main:

entries = {}
entryHashes = {}
compteHashes = {}
compteFiles = {}

for csvFile in files:
    compte, date, minDate, maxDate, solde, entries[csvFile] = readCSV(csvFile)
    #print("COMPTE=" + compte)
    if not compte in compteFiles:
        compteHashes[compte] = {}
        compteFiles[compte] = []
        
    compteFiles[compte].append( (csvFile, compte, date, minDate, maxDate, solde, entries[csvFile]) )
        
    print("<{}> -> <{}> compte {} date {} solde {}".format(minDate, maxDate, compte, date, solde))
    print()

print("------------------------")
print()

#file_handles = []
#try:
#    for file in file_list:
#        file_handles.append(open(file))
    # Do some calculations with open files
#finally:
#    for fh in file_handles:
#        fh.close()
        
def getHashesSeen(hashFile):
    if not os.path.exists(hashFile):
        return {}
    
    hashesSeen = {}
    lines=0
    with open(hashFile, 'r') as fh:
        for hashline in fh.readlines():
            hashline = hashline.rstrip()
            #print("line=<" + hashline + "> len=" + str(len(hashline)))
            if len(hashline) == 0:
                continue
            if hashline[0] == '#':
                continue
            if hashline[0] != '<' or hashline.find('>') < 0:
                die("Bad syntax in line <{}>".format(hashline))
        
            lines += 1
            hash=hashline[1:hashline.find('>')]
            hashesSeen[hash]=True
        
    ohashfh.close()
    
    print("Read {} lines from hashfile {}".format(lines, hashFile))
    return hashesSeen

for compte in compteFiles:
    print("")
    print(" -- Analyzing compte {} {}".format(compte, str(comptes[compte])))
    
    # Make sure files for a compte are in date order:
    #sorted(student_tuples, key=lambda student: student[2])   # sort by age
    #sorted(compteFiles[compte], key=lambda a: getDateNumber(a[2]))
    sorted(compteFiles[compte], key=cmp_to_key(compareMinMaxDates))
    
    compteHashes[compte] = {}

    baseFileName = comptes[compte][1]
    
    tmpdir = 'BanquePostale.CSV/tmp/'
    
    deltaFile = tmpdir + baseFileName + '.delta'
    hashFile  = tmpdir + baseFileName + '.hashes'
    
    now = str(datetime.datetime.now()) 

    hashesSeen = getHashesSeen(hashFile)
    ohashfh = open(hashFile, 'a')
    ohashfh.write("\n# -------- {} \n".format(now))

    odeltafh = open(deltaFile, 'a')
    odeltafh.write("\n# -------- {} \n".format(now))

    for fileTpl in compteFiles[compte]:
        
        #print()
        (csvFile, compte, date, minDate, maxDate, solde, entries) = fileTpl
        print("-- FILE - {} ---------------------------------------------------".format(csvFile))
        print("{} entries: <{}> -> <{}> compte {} date {} solde {}".format(len(entries), minDate, maxDate, compte, date, solde))
        #continue
        ohashfh.write("# -------- {} / {} entries\n".format(csvFile, len(entries)))
        odeltafh.write("# -------- {} / {} entries\n".format(csvFile, len(entries)))
        
        localDups = 0
        dups = 0
        
        for entry in entries:
            (hash, row) = entry
            
            # Date;Libelle;Montant(EUROS)
            if hash in hashesSeen:
                dups += 1
                #print("DUP HASH")
                continue
            if hash in compteHashes[compte]:
                localDups += 1
                #print("DUP LOCAL HASH")
                continue
                
            compteHashes[compte][hash] = True
            hashesSeen[hash] = True
            #print("{}<{}>({}): {}".format(row[1], hash, row[0], row[2]))
                
            matched = False
            tags = 'UNKNOWN'
            commercant = 'UNKNOWN'
            info = row[1]

            for regex in REGEXES:
                p = re.compile(regex)
                m = p.match(row[1])
                if m != None:
                    matched = True
                    fields = REGEXES[regex]
                    commercant = fields[0].lower().capitalize()
                    tags = fields[1].lower().capitalize()
                    info = fields[2]
                    #print("<{}>   <{}>    <{}>".format(m.group(0), m.group(1), m.group(2)))
                    break
                        
            if info != '':
                info = '{' + info.replace(':', '_').lower().capitalize() + '}'
            
            OP_LINE="{} {}{}({}): {}".format(tags, commercant, info, row[0], row[2])
            OP_LINE = " ".join(OP_LINE.split())
            odeltafh.write(OP_LINE + "\n")
            
            HASHED_LINE="<{}>{} {}{}({}): {}".format(hash, tags, commercant, info, row[0], row[2])
            HASHED_LINE = " ".join(HASHED_LINE.split())
            ohashfh.write(HASHED_LINE + "\n")

        if localDups or dups:
            dupsInfo = '# '
            if localDups > 0:
                dupsInfo += "Ignored {} local dups".format(localDups)
            if localDups and dups:
                dupsInfo += ", "
            if dups > 0:
                dupsInfo += "Ignored {} dups".format(dups)
            print(dupsInfo)
            odeltafh.write(dupsInfo + "\n")
            ohashfh.write(dupsInfo + "\n")

    odeltafh.close()
    ohashfh.close()

# NEXT op to BASEFILENAME.temp.mon

DEBUG <Parsing CSV file <BanquePostale.CSV/0992100C0281436075147363.csv>>
<2015-03-02> -> <2015-07-03> compte 0992100C028 date 2015-03-02 solde 4415.68

DEBUG <Parsing CSV file <BanquePostale.CSV/0992100C0281440321506749.csv>>
<2015-07-24> -> <2015-08-21> compte 0992100C028 date 2015-07-24 solde 1008.15

DEBUG <Parsing CSV file <BanquePostale.CSV/1063284B0281440321601474.csv>>
<2015-06-15> -> <2015-08-05> compte 1063284B028 date 2015-06-15 solde 3615.02

DEBUG <Parsing CSV file <BanquePostale.CSV/1063284B0281440837564490.csv>>
<2015-06-15> -> <2015-08-28> compte 1063284B028 date 2015-06-15 solde 5549.75

------------------------


 -- Analyzing compte 0992100C028 ['CJointe', 'FR_EU_LaPoste_CJointe']
-- FILE - BanquePostale.CSV/0992100C0281436075147363.csv ---------------------------------------------------
162 entries: <2015-03-02> -> <2015-07-03> compte 0992100C028 date 2015-03-02 solde 4415.68
-- FILE - BanquePostale.CSV/0992100C0281440321506749.csv ----------------------------------

In [22]:
a="a string"
b=a.rstrip("gn")
print("b=" + b)

import re
s = "The   fox jumped   over    the log."
re.sub("\s\s+" , " ", s)


b=a stri


'The fox jumped over the log.'

In [ ]:
string = "This Is A String and this also is a String, and this is also a String"
print(string.replace("String","Text"))

string = string.lower().capitalize()
print(string)


string = string.lower()
print(string)

string = string.capitalize()
print(string)

In [ ]:
%tb


[TOP](#TOP)
---
# MOOCS.py

[TOP](#TOP)
---
# %ruby

In [ ]:
%%ruby

puts "Did it take long to start"

In [ ]:
%%ruby

puts "Did it take long to start"

In [ ]:
%%ruby

puts "Did it take long to start"

[TOP](#TOP)
---
# Magics

In [ ]:
%lsmagic


[TOP](#TOP)
---
# APIS_OSTACK

In [ ]:
# First try some cli stuff (to make sure certificates are OK)

# Then do some Python API code


[TOP](#TOP)
---
# APIS_DOCKER

In [ ]:
# First try some cli stuff (to make sure certificates are OK)

# Then some Python API code

# Then some Go / ...

# Then some visualizations

# https://readthedocs.org/projects/docker-py/
    
from docker import Client
#c = Client(base_url='unix://var/run/docker.sock')
#
c = Client(base_url='http://127.0.0.1:8876')
print(c)
c.info()
    

In [ ]:
%%svg
<svg xmlns="http://www.w3.org/2000/svg">
<metadata>Generated by IcoMoon</metadata>
<defs>
<font id="icons" horiz-adv-x="1024">
<font-face units-per-em="1024" ascent="960" descent="-64" />
<missing-glyph horiz-adv-x="1024" />
<glyph unicode="&#x20;" d="" horiz-adv-x="512" />
<glyph unicode="&#xe000;" d="M0 960v-204.8h614.4v204.8h-614.4zM0 550.4v-204.8h1024v204.8h-1024zM0 140.8v-204.8h409.6v204.8h-409.6zM614.4 140.8v-204.8h409.6v204.8h-409.6zM819.2 960v-204.8h204.8v204.8h-204.8z" />
<glyph unicode="&#xe001;" d="M842.688-64c-0.8 459.232-401.504 834.528-842.688 835.392v188.608c541.312 0 1024-460.608 1024-1024h-181.312zM687.2-63.936h-181.216c0.608 112.768-56.672 238.976-147.328 333.472-90.144 95.264-249.92 155.296-358.208 154.72v188.544c338.080-7.296 679.872-326.624 686.752-676.736zM128.512 192.096c70.624 0 127.872-57.312 127.872-128 0-70.656-57.248-127.968-127.872-127.968s-127.872 57.312-127.872 127.968c0 70.688 57.248 128 127.872 128z" />
<glyph unicode="&#xe002;" d="M1024 765.582c-37.676-16.708-78.164-28.002-120.66-33.080 43.372 26 76.686 67.17 92.372 116.23-40.596-24.078-85.556-41.56-133.41-50.98-38.32 40.83-92.922 66.34-153.346 66.34-116.022 0-210.088-94.058-210.088-210.078 0-16.466 1.858-32.5 5.44-47.878-174.6 8.764-329.402 92.4-433.018 219.506-18.084-31.028-28.446-67.116-28.446-105.618 0-72.888 37.088-137.192 93.46-174.866-34.438 1.092-66.832 10.542-95.154 26.278-0.020-0.876-0.020-1.756-0.020-2.642 0-101.788 72.418-186.696 168.522-206-17.626-4.8-36.188-7.372-55.348-7.372-13.538 0-26.698 1.32-39.528 3.772 26.736-83.46 104.32-144.206 196.252-145.896-71.9-56.35-162.486-89.934-260.916-89.934-16.958 0-33.68 0.994-50.116 2.94 92.972-59.61 203.402-94.394 322.042-94.394 386.422 0 597.736 320.124 597.736 597.744 0 9.108-0.206 18.168-0.61 27.18 41.056 29.62 76.672 66.62 104.836 108.748z" />
<glyph unicode="&#xe003;" d="M0.806 90.336c-0.244 2.532-0.452 5.070-0.584 7.63 0.13-2.56 0.34-5.098 0.584-7.63zM235.908 394.852c92.010-2.738 153.734 92.698 137.862 213.198-15.894 120.48-103.396 217.168-195.408 219.922-92.026 2.73-153.74-89.482-137.852-210 15.882-120.468 103.352-220.374 195.398-223.12zM1024 704v85.31c0 93.88-76.782 170.69-170.658 170.69h-682.656c-92.276 0-168.012-74.232-170.564-165.926 58.362 51.386 139.324 94.316 222.874 94.316 89.304 0 357.244 0 357.244 0l-79.948-67.618h-113.268c75.13-28.804 115.156-116.124 115.156-205.722 0-75.248-41.81-139.954-100.888-185.968-57.644-44.902-68.572-63.708-68.572-101.878 0-32.578 61.746-88 94.032-110.788 94.382-66.538 124.916-128.312 124.916-231.456 0-16.428-2.042-32.83-6.066-48.96h307.742c93.874 0 170.656 76.75 170.656 170.69v533.308h-192v-191.998h-64v192h-191.998v64h191.998v192h64v-192h192zM185.886 194.064c21.614 0 41.422 0.59 61.936 0.59-27.146 26.334-48.626 58.6-48.626 98.38 0 23.608 7.564 46.336 18.134 66.52-10.782-0.77-21.79-0.994-33.126-0.994-74.356 0-137.506 24.076-184.204 63.854v-67.242l0.006-201.73c53.44 25.374 116.888 40.622 185.88 40.622zM3.42 72.742c-1.112 5.458-1.966 11.006-2.542 16.634 0.574-5.628 1.43-11.176 2.542-16.634zM455.45 7.154c-15.058 58.806-68.454 87.964-142.888 139.568-27.072 8.732-56.894 13.874-88.894 14.208-89.618 0.964-173.108-34.942-220.216-88.372 15.92-77.706 85.034-136.558 167.234-136.558h286.444c1.816 11.128 2.696 22.632 2.696 34.432 0 12.534-1.534 24.792-4.376 36.722z" />
<glyph unicode="&#xe004;" d="M853.344 960h-682.66c-93.872 0-170.684-76.814-170.684-170.688v-682.626c0-93.938 76.812-170.686 170.684-170.686l682.66 0.002c93.876 0 170.656 76.746 170.656 170.688v682.622c0 93.874-76.782 170.688-170.656 170.688zM870.592 448h-166.592v-448h-192v448h-92.526v146.564h92.526v95.186c0 129.342 55.792 206.25 207.87 206.25h175.244v-158.57h-143.13c-42.482 0.070-47.752-22.152-47.752-63.512l-0.232-79.354h192l-25.408-146.564z" />

In [ ]:
%config InlineBackend.figure_format = 'svg'

# svg examples:
#   http://www.w3schools.com/svg/default.asp
# But see %%html and %%svg below ...

#<svg width="100" height="100">
#  <circle cx="50" cy="50" r="40" stroke="green" stroke-width="4" fill="yellow" />
#</svg>

In [ ]:
%config InlineBackend.figure_format = 'svg'
#<circle cx="50" cy="50" r="40" stroke="green" stroke-width="4" fill="yellow" />


In [ ]:
%%html
<html>
<head>
</head>
<body>

    <!-- First inline SVG with inline style block -->
    <svg width="10cm" height="5cm" viewBox="0 0 1000 500">
    <defs>
    <style><![CDATA[

        rect {
            fill: red;
            stroke: blue;
            stroke-width: 3
        }
        ]]></style>
    </defs>
    <rect x="200" y="100" width="600" height="300"/>
    </svg>

    <!-- Second SVG -->
    <svg width="10cm" height="5cm" viewBox="0 0 1000 500">
    <rect x="200" y="100" width="600" height="300"/>
    </svg>

</body>
</html>

In [ ]:
%%svg
<svg width="10cm" height="5cm" viewBox="0 0 1000 500">
    <rect x="200" y="100" width="600" height="300"/>
</svg>